In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus


p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

## weighted triplets?


Properties we want:

* higher score when there are a lot of "major" isoforms (0.33, 0.33, 0.33)
* number should be bigger the more isoforms there are
* maximum value (scaled)
* 1 isoform --> 1

https://en.wikipedia.org/wiki/Harmonic_mean#Weighted_harmonic_mean

https://en.wikipedia.org/wiki/Entropy

https://en.wikipedia.org/wiki/Gini_coefficient

https://academic.oup.com/bib/article/18/2/205/2562739

Try Gini and Tau from this paper

In [2]:
def compute_hg(pis):
    val = 0
    for pi in pis:
        val += pi*math.log2(pi)
    val = -1*val
    return val

In [3]:
def compute_tau(x):
    x_hat = max(x)
    n = len(x)
    tau = 0
    for i in range(n):
        tau += (1-x_hat)
    tau = tau/(n-1)
    return tau

In [4]:
def compute_gini(pis):
    n = len(pis)
    pis = sorted(pis)
    gini = ((n+1)/n) - \
        (((2*sum([(n+1-i)*xi for i,xi in enumerate(pis)])) / \
          (n*sum([xi for xi in pis])))) 
    return gini

In [6]:
# sorted(pis)

In [7]:
def compare_metrics(pis):
    print('hg: {}'.format(compute_hg(pis)))
    print('tau: {}'.format(compute_tau(pis)))
    print('gini: {}'.format(compute_gini(pis)))

In [8]:
pis = [.1 for i in range(10)]
compare_metrics(pis)

hg: 3.321928094887362
tau: 1.0000000000000002
gini: -0.20000000000000018


In [9]:
pis = [.3 for i in range(3)]
compare_metrics(pis)

hg: 1.5632690347495855
tau: 1.0499999999999998
gini: -0.6666666666666667


In [10]:
pis = [.9, .1]
compare_metrics(pis)

hg: 0.4689955935892812
tau: 0.19999999999999996
gini: -0.6000000000000001


In [11]:
pis = [.9]+[.01 for i in range(10)]
print(sum(pis))
compare_metrics(pis)

1.0
hg: 0.8011884030780172
tau: 0.10999999999999996
gini: 0.6272727272727272


In [12]:
pis = [.9]+[.05 for i in range(2)]
print(sum(pis))
compare_metrics(pis)

1.0
hg: 0.5689955935892812
tau: 0.14999999999999997
gini: -0.10000000000000009
